## Simulating sequences

Full simulation pipeline to get DNA sequences via msprime and seq-gen, as well as to create the site-count matrix.

In [23]:
import msprime
import numpy as np
import dendropy
from dendropy.interop import seqgen

In [11]:
samplenum = 2000 # how many simulation samples
# make list of migration rates
mig_rate = np.append(np.random.gamma(shape = 1,  # make half of the rates positive and the other half zero
                                     scale=.01,
                                     size = int(samplenum/2.)),
                     [0]*(int(samplenum/2.)))
np.random.shuffle(mig_rate)

# select window width 
win_width = [.05]*samplenum # if varying: np.random.uniform(0.03,0.1,size = 1000000)
# select window center 
win_center = [.5]*samplenum # if varying: np.random.uniform(0.1, 0.9, size = 1000000)
# get number of replicates
#num_reps_list = [10000]*samplenum # if varying: (5000+np.random.randint(1,15000,1000000))*2

In [267]:
sample = 0
length = 10000
Ne = 1000000
mutation_rate = 7e-8
num_replicates = 1
recombination_rate = 1e-8

# Four tips of the tree
population_configurations = [msprime.PopulationConfiguration(sample_size=1,initial_size=250000),
                            msprime.PopulationConfiguration(sample_size=1,initial_size=250000),
                            msprime.PopulationConfiguration(sample_size=1,initial_size=250000),
                            msprime.PopulationConfiguration(sample_size=1,initial_size=250000),]

# No migration initially
migration_matrix = [[0.,0.,0.,0.],
                    [0.,0.,0.,0.],
                    [0.,0.,0.,0.],
                    [0.,0.,0.,0.],]

# Define demographic events for msprime
demographic_events = [msprime.MigrationRateChange(time=0,
                                                  rate=0),
                     msprime.MigrationRateChange(time=(win_center[sample]-win_width[sample]/2.)*4*Ne,
                                                 rate=mig_rate[sample],
                                                 matrix_index=[0,2]),
                     msprime.MigrationRateChange(time=(win_center[sample]+win_width[sample]/2.)*4*Ne,
                                                 rate=0),
                     msprime.MassMigration(destination=1,
                                           source=0,
                                           time=1.0*4*Ne,
                                           proportion=1),
                     msprime.MassMigration(destination=1, 
                                           source=2,
                                           time=1.2*4*Ne,
                                           proportion=1),
                     msprime.MassMigration(destination=1, 
                                           source=3,
                                           time=1.5*4*Ne,
                                           proportion=1)
                     ]

# Our msprime simulation:
simulation = msprime.simulate(length=length,
                 Ne=Ne,
                 mutation_rate=mutation_rate,
                 num_replicates=num_replicates,
                 recombination_rate=recombination_rate,
                 population_configurations=population_configurations,
                 migration_matrix = migration_matrix,         
                 demographic_events=demographic_events)


In [268]:
fullseq = None
seqlength = 5
all_labels = np.empty(shape = [0,1])
all_seqs = np.empty(shape = [0,seqlength])
for currsim in simulation:
    simtrees = currsim.trees()
    for currtree in simtrees:
        phylo = dendropy.TreeList.get(data = currtree.newick(),schema='newick')
        s = seqgen.SeqGen()
        s.seq_len = seqlength
        s.scale_branch_lens = 0.00001 # will have to consider how best to use this
        dnamat=s.generate(phylo)
        charmat=dnamat.char_matrices[0]
        for lab,seq in charmat.items():
            all_labels = np.append(all_labels,lab.label)
            all_seqs = np.vstack([all_seqs,np.array(seq.symbols_as_list())])

In [269]:
seqs = np.vstack([np.concatenate(all_seqs[all_labels == '1']),np.concatenate(all_seqs[all_labels == '2']),np.concatenate(all_seqs[all_labels == '3']),np.concatenate(all_seqs[all_labels == '4'])])

In [270]:
seqs

array([['A', 'T', 'C', ..., 'C', 'T', 'T'],
       ['A', 'T', 'A', ..., 'C', 'T', 'T'],
       ['A', 'T', 'C', ..., 'C', 'A', 'G'],
       ['A', 'C', 'C', ..., 'C', 'A', 'T']],
      dtype='|S32')